In [1]:
#インポート
from bs4 import BeautifulSoup as bs
import re
import time
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
import unicodedata
import openai
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()
api_version="2023-12-01-preview"
engine = 'gpt-4o-mini'
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY_GPT4OMINI"),  
    api_version=api_version,
    azure_endpoint = os.getenv("OPENAI_API_BASE_GPT4OMINI")
    )
data = pd.read_csv("../data.csv")

In [71]:
class GPT4o_mini():
    def __init__(self, query, table_id, path):
        self.query = query
        self.table_id = table_id
        self.path = path

    def edit_query(self):
        year = re.search('20(\d)+', self.query).group()
        if year == '2023':
            return re.sub(f'{year}年', f'最新', self.query)
        elif year == '2022':
            return re.sub(f'{year}年', f'前期', self.query)
        elif year == '2021':
            return re.sub(f'{year}年', f'前々期', self.query)
        elif year == '2020':
            return re.sub(f'{year}年', f'3期前', self.query)
        elif year == '2019':
            return re.sub(f'{year}年', f'4期前', self.query)

    #html内のtableデータを抽出してDataFrameに変換
    def get_cellid(self,):
        doc_id = self.table_id.split("-")[0]
        file = self.table_id.split("-")[1]
        
        with open(f'{self.path}/{doc_id}/{file}.html', 'r') as f:
            soup = bs(f, 'html.parser')
            f.close()
        
        html_tag = soup.find_all(['table',])
        for _, table in enumerate(html_tag):
            if table.get('table-id') == self.table_id:
                break

        max_row =0
        for row in table.find_all('tr'):
            tmp_row = 0
            for cell in row.find_all(['td', 'th']):
                tmp_row += 1
                if cell.get('colspan'):
                    tmp_row += int(cell.get('colspan'))-1
            if tmp_row > max_row:
                max_row = tmp_row

        df = pd.DataFrame([['?']*max_row])
        row_index = 0
        for row in table.find_all('tr'):
            col_index = 0
            for cell in row.find_all(['td', 'th']):
                try:
                    while df.iloc[row_index, col_index] != '?':
                        col_index += 1
                except:
                    df = pd.concat([df, pd.DataFrame([['?']*len(df.columns)])])
                    
                cell_text = unicodedata.normalize('NFKC', cell.get('cell-id')).replace('\n', '').replace(' ', '')
                cell_text = cell_text.replace("△", "-")   
                if cell.get('rowspan'):
                    rowspan = int(cell.get('rowspan'))
                    df = pd.concat([df, pd.DataFrame([['?']*len(df.columns)]*(rowspan))])
                    df.iloc[row_index:row_index+rowspan, col_index] = cell_text
                elif cell.get('colspan'):
                    colspan = int(cell.get('colspan'))                
                    df.iloc[row_index, col_index:col_index+colspan] = cell_text
                    col_index += colspan-1
                
                else:   
                    df.iloc[row_index, col_index] = cell_text
                col_index += 1
            row_index += 1

        df = df.drop_duplicates()
        df = df.T.drop_duplicates().T
        df.columns = range(len(df.columns))
        df.index = range(len(df))
        return df

    
    #html内のtableデータを抽出してDataFrameに変換
    def html_to_table(self):
        doc_id = self.table_id.split("-")[0]
        file = self.table_id.split("-")[1]
        
        with open(f'{self.path}/{doc_id}/{file}.html', 'r') as f:
            soup = bs(f, 'html.parser')
            f.close()
        dfs = pd.read_html(f'{self.path}/{doc_id}/{file}.html')
        tags = soup.find_all(['table',])
        i = 0
        for tag in tags:
            if tag.get('table-id') == self.table_id:
                break
            else:
                i += 1

        df_ids = self.get_cellid()
        df_ids.loc[0:1] = dfs[i].loc[0:1]
        df_ids[[0, 1]] = dfs[i][[0, 1]]
        return dfs[i], df_ids

    #ChatGPTに質問を投げて、表から回答を抽出する
    def rewrite_query(self, ):
        prompt =f"次の手順に従て与えられた質問文を書き換えてください。\n1.アルファベット部分を日本語に変換してください\n2.必要な情報のみを抽出して質問文を簡略化してください。 \n##Question\n{self.edit_query()}"
        messages=[
            {'role':'system', "content": 'あなたは表からデータを抽出するタスクを任されています。'},
            {"role": "user", "content": prompt}, 
            ]
        
        response = client.chat.completions.create(
            model= engine,
            messages = messages,
            temperature=0,
            max_tokens=2048,
            top_p=0.1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )
        return response.choices[0].message.content


    #ChatGPTに質問を投げて、表から回答を抽出する
    def answer(self, ):
        table_md = self.html_to_table()[0].to_markdown(index=False)
        display(self.html_to_table()[0])
        rewrite_query = self.rewrite_query()
        print(rewrite_query)
        prompt =f"""
        表から質問の回答を抽出してください。抽出できない場合-を出力してください。
        #TABLE
        {table_md}
        #Question
        {rewrite_query}"""

        messages=[
            {"role": "user", "content": prompt}, 
            ]
        tools = [
                {
                    "type":"function",
                    "strict":"true",
                    "function": {
                        "name": "table_answer_extractor",
                        "discription": "Extract answer from table",
                        "parameters":{
                            "type": "object",
                            "properties": {
                                "value": {
                                    "type": "string",
                                    "description": "Value of the cell"
                                    },
                                "unit": {
                                    "type": "string",
                                    "description": "Unit of the cell"
                                    },
                                },
                            "required":['question', "value", "unit"]
                            },
                        }
                }
            ] 
        
        response = client.chat.completions.create(
            model= engine,
            messages = messages,
            tools = tools,
            temperature=0,
            max_tokens=2048,
            top_p=0.1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )
        response_message = response.choices[0].message

    # Handle function calls
        if response_message.tool_calls:
            for tool_call in response_message.tool_calls:
                function_args = json.loads(tool_call.function.arguments)
        return function_args
        

In [72]:
def normalize(dct):
    value = unicodedata.normalize('NFKC', dct['value']).replace('△', '-')
    unit = unicodedata.normalize('NFKC', dct['unit'])

    if value =='-':
        return value
    elif '百万' in unit:
        return value.replace(',', '').replace(' ', '')+'000000'
    elif '%' in unit:
        return str(float(value.replace(' ', ''))/100)
    elif '千' in unit:
        return value.replace(',', '').replace(' ', '')+'000'
    else:
        return value.replace(',', '')

In [73]:
data.query('question_number == "question_valid63"')

,question_number,question,doc_id,table_id,cell_id,value,flag,ja_question,tmp_question,words
12911,question_valid63,テクノプロ・ホールディングス株式会社の2020年時点のNonConsolidatedMemb...,S100RXK0,S100RXK0-0101010-tab3,S100RXK0-0101010-tab3-r20c4,179,valid,テクノプロ・ホールディングス株式会社の2020年時点の非連結又は個別における従業員数は?,2020年時点の非連結又は個別における従業員数は?,"['従業員数', 'NonConsolidatedMember']"


In [74]:
path = '../valid/reports'
table_id, query, value, cell_id = data.query('question_number == "question_valid90"')[['table_id', 'tmp_question', 'value', 'cell_id']].values[0]
print(table_id ,query)
print(value)
print(cell_id)
#df, df_id = GPT4o_mini(query, table_id, path).answer()
#display(df_id)
df = GPT4o_mini(query, table_id, path).answer()
display(df)


S100RXK0-0105010-tab71 2023年における当期利益(△損失)の親会社の所有者は?
15365000000
S100RXK0-0105010-tab71-r17c4


,0,1,2,3
0,NaN,NaN,NaN,（単位：百万円）
1,NaN,注記,前連結会計年度 (自 2021年７月１日 至 2022年６月30日),当連結会計年度 (自 2022年７月１日 至 2023年６月30日)
2,売上収益,５、24,178756,199851
3,売上原価,25,131897,146948
4,売上総利益,NaN,46858,52903
5,販売費及び一般管理費,26,27819,31523
6,その他の収益,29,2387,582
7,その他の費用,29,784,124
8,営業利益,NaN,20641,21838
9,金融収益,28,460,196


##質問
最新の当期利益(△損失)の親会社の所有者は誰ですか？


{'value': '15365', 'unit': '百万円'}

In [ ]:
path = '../valid/reports'
table_id, query, value = data.query('flag == "valid"')[['table_id', 'tmp_question', 'value']].values[10]
print(table_id ,query)
print(value)
GPT4o_mini(query, table_id, path).answer()

In [291]:
from tqdm import tqdm
path = '../valid/reports'
answer_sheet = {}
for index, row in data.query('flag =="valid"')[:20].iterrows():
    table_id, query, value = row[['table_id', 'tmp_question', 'value']]
    answer = GPT4o_mini(query, table_id, path).answer()
    print(row['question_number'], value)
    answer_sheet[row['question_number']] = normalize(answer)

,0,1,2,3,4,5,6
0,回次,回次,第172期,第173期,第174期,第175期,第176期
1,決算年月,決算年月,2020年３月,2021年３月,2022年３月,2023年３月,2024年３月
2,売上高,(百万円),672314,643736,670350,806453,799760
3,経常利益,(百万円),1312,18380,34089,67897,69733
4,親会社株主に帰属する 当期純利益又は親会社株主に帰属する当期純損失（△）,(百万円),"△ 38,510","△ 5,369",39101,40891,51011
5,包括利益,(百万円),"△ 54,950",9948,59274,58944,84725
6,純資産額,(百万円),172115,184483,243657,294384,366582
7,総資産額,(百万円),576090,569124,611526,656785,723867
8,１株当たり純資産額,(円),552.30,591.24,799.76,980.91,1236.73
9,１株当たり当期純利益 又は当期純損失（△）,(円),△ 136.58,△ 19.50,141.85,148.27,184.96


##質問
前々期の自己資本利益率は?
question_valid1 0.2040


,0,1,2,3,4,5,6
0,回次,回次,第172期,第173期,第174期,第175期,第176期
1,決算年月,決算年月,2020年３月,2021年３月,2022年３月,2023年３月,2024年３月
2,売上高,(百万円),672314,643736,670350,806453,799760
3,経常利益,(百万円),1312,18380,34089,67897,69733
4,親会社株主に帰属する 当期純利益又は親会社株主に帰属する当期純損失（△）,(百万円),"△ 38,510","△ 5,369",39101,40891,51011
5,包括利益,(百万円),"△ 54,950",9948,59274,58944,84725
6,純資産額,(百万円),172115,184483,243657,294384,366582
7,総資産額,(百万円),576090,569124,611526,656785,723867
8,１株当たり純資産額,(円),552.30,591.24,799.76,980.91,1236.73
9,１株当たり当期純利益 又は当期純損失（△）,(円),△ 136.58,△ 19.50,141.85,148.27,184.96


最新の自己資本利益率は?
question_valid2 0.1668


,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024年３月31日現在,2024年３月31日現在,2024年３月31日現在
1,区分,株式の状況（１単元の株式数 100 株）,株式の状況（１単元の株式数 100 株）,株式の状況（１単元の株式数 100 株）,株式の状況（１単元の株式数 100 株）,株式の状況（１単元の株式数 100 株）,株式の状況（１単元の株式数 100 株）,株式の状況（１単元の株式数 100 株）,株式の状況（１単元の株式数 100 株）,単元未満株式の状況 （株）
2,区分,政府及び地方公共団体,金融機関,金融商品取引業者,その他の法人,外国法人等,外国法人等,個人その他,計,単元未満株式の状況 （株）
3,区分,政府及び地方公共団体,金融機関,金融商品取引業者,その他の法人,個人以外,個人,個人その他,計,単元未満株式の状況 （株）
4,株主数（人）,－,48,40,252,368,64,25141,25913,－
5,所有株式数 （単元）,－,1334761,122521,145408,763136,695,590510,2957031,160321
6,所有株式数の割合（％）,－,45.14,4.14,4.92,25.81,0.02,19.97,100,－


...

##質問
前期の連結決算における営業利益は？
question_valid26 70163000000


,0,1,2
0,NaN,NaN,(単位：百万円)
1,NaN,前連結会計年度 (自 2022年４月１日 至 2023年３月31日),当連結会計年度 (自 2023年４月１日 至 2024年３月31日)
2,売上高,"※１ 806,453","※１ 799,760"
3,売上原価,"※３ , ※４ 635,434","※３ , ※４ 629,053"
4,売上総利益,171019,170707
5,販売費及び一般管理費,NaN,NaN
6,荷造運搬費,23416,19489
7,人件費,"※２ 42,858","※２ 45,044"
8,その他,"※３ 34,582","※３ 36,690"
9,販売費及び一般管理費合計,100856,101223


最新の連結決算における特別損失の子会社清算損は何ですか？
question_valid30 －


,0,1,2,3,4,5
0,NaN,NaN,NaN,NaN,(単位：百万円),(単位：百万円)
1,NaN,株主資本,株主資本,株主資本,株主資本,株主資本
2,NaN,資本金,資本剰余金,利益剰余金,自己株式,株主資本合計
3,当期首残高,53076,28054,126530,"△ 10,575",197085
4,当期変動額,NaN,NaN,NaN,NaN,NaN
5,剰余金の配当,NaN,NaN,"△ 6,357",NaN,"△ 6,357"
6,親会社株主に帰属する当期純利益,NaN,NaN,40891,NaN,40891
7,自己株式の取得,NaN,NaN,NaN,△ 2,△ 2
8,自己株式の処分,NaN,NaN,NaN,108,108
9,非支配株主との取引に係る親会社の持分変動,NaN,"△ 1,548",NaN,NaN,"△ 1,548"


##質問
前期の連結決算の資本剰余金の連結範囲の変動は？
question_valid32 -846000000


In [292]:
gold_sd = dict(data.query('flag =="valid"')[['question_number', 'value']].values)

In [293]:
total = 0
correct = 0

for key in answer_sheet.keys():
    total += 1
    try:
        if float(gold_sd[key]) == float(answer_sheet[key]):
            correct += 1
        else:
            print(f'{key}, TRUE:{gold_sd[key]}, PRED:{answer_sheet[key]}')
            pass
    except:
        if unicodedata.normalize('NFKC',gold_sd[key]) == unicodedata.normalize('NFKC',answer_sheet[key]):
            correct += 1
        else:
            print(f'{key}, TRUE:{gold_sd[key]}, PRED:{answer_sheet[key]}')
            pass
        
print(correct/total)

question_valid10, TRUE:－, PRED:295863421
question_valid14, TRUE:190000000, PRED:1000000
question_valid15, TRUE:1000000, PRED:-
question_valid21, TRUE:-64741000000, PRED:-258324000000
question_valid32, TRUE:-846000000, PRED:nan000000
0.75


In [77]:
data.query('flag == "train" and words == @words')

,question_number,question,doc_id,table_id,cell_id,value,flag,ja_question,tmp_question,words
0,question_train3,株式会社アインホールディングスの2021年における「１株当たり当期純利益又は当期純損失（△）...,S100RHMW,S100RHMW-0101010-tab2,S100RHMW-0101010-tab2-r10c5,189.04,train,株式会社アインホールディングスの2021年における経営指標等の1株当たり当期純利益又は当期純...,2021年における経営指標等の自己資本利益率は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
1,question_train4,株式会社アインホールディングスの2023年における「１株当たり当期純利益又は当期純損失（△）...,S100RHMW,S100RHMW-0101010-tab2,S100RHMW-0101010-tab2-r10c7,262.87,train,株式会社アインホールディングスの2023年における経営指標等の1株当たり当期純利益又は当期純...,2023年における経営指標等の自己資本利益率は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
142,question_train285,レーザーテック株式会社の2023年における「１株当たり当期純利益又は当期純損失（△）、経営指...,S100RWZQ,S100RWZQ-0101010-tab2,S100RWZQ-0101010-tab2-r10c7,511.89,train,レーザーテック株式会社の2023年における経営指標等の1株当たり当期純利益又は当期純損失(△)は?,2023年における提出会社の保有目的が純投資目的以外の目的である投資株式の非上場株式以外の株...,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
802,question_train1606,キヤノンマーケティングジャパン株式会社の2021年における「１株当たり当期純利益又は当期純損...,S100T53R,S100T53R-0101010-tab2,S100T53R-0101010-tab2-r10c5,226.88,train,キヤノンマーケティングジャパン株式会社の2021年における経営指標等の1株当たり当期純利益又...,2024年の連結決算における税引前当期純利益又は税引前当期純損失(△)は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
980,question_train1958,株式会社シマノの2021年における「１株当たり当期純利益又は当期純損失（△）、経営指標等」は？,S100T5J9,S100T5J9-0101010-tab2,S100T5J9-0101010-tab2-r10c5,1252.62,train,株式会社シマノの2021年における経営指標等の1株当たり当期純利益又は当期純損失(△)は?,2024年時点の普通株式における株式の総数等の発行済株式の種類は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
981,question_train1959,株式会社シマノの2023年における「１株当たり当期純利益又は当期純損失（△）、経営指標等」は？,S100T5J9,S100T5J9-0101010-tab2,S100T5J9-0101010-tab2-r10c7,676.77,train,株式会社シマノの2023年における経営指標等の1株当たり当期純利益又は当期純損失(△)は?,2024年における監査公認会計士等の監査証明業務に基づく報酬-連結子会社は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
1041,question_train2082,株式会社パイロットコーポレーションの2021年における「１株当たり当期純利益又は当期純損失（...,S100T4Z1,S100T4Z1-0101010-tab2,S100T4Z1-0101010-tab2-r10c5,361.81,train,株式会社パイロットコーポレーションの2021年における経営指標等の1株当たり当期純利益又は当...,2024年の連結決算におけるその他の包括利益のその他有価証券評価差額金(税引後)は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
1835,question_train3666,株式会社三井ハイテックの2022年における「１株当たり当期純利益又は当期純損失（△）、経営指...,S100TAPP,S100TAPP-0101010-tab2,S100TAPP-0101010-tab2-r10c5,322.24,train,株式会社三井ハイテックの2022年における経営指標等の1株当たり当期純利益又は当期純損失(△)は?,2024年の個別決算の非連結又は個別における法人税等は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
1836,question_train3667,株式会社三井ハイテックの2024年における「１株当たり当期純利益又は当期純損失（△）、経営指...,S100TAPP,S100TAPP-0101010-tab2,S100TAPP-0101010-tab2-r10c7,425.30,train,株式会社三井ハイテックの2024年における経営指標等の1株当たり当期純利益又は当期純損失(△)は?,2024年時点における表紙の提出先は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
2087,question_train4172,株式会社エービーシー・マートの2024年における「１株当たり当期純利益又は当期純損失（△）、...,S100TIG3,S100TIG3-0101010-tab2,S100TIG3-0101010-tab2-r10c7,161.58,train,株式会社エービーシー・マートの2024年における経営指標等の1株当たり当期純利益又は当期純損...,2023年のその他の資本の構成要素における当期利益(△損失)は?,['１株当たり当期純利益又は当期純損失（△）、経営指標等']
